In [59]:
import pandas as pd
import matplotlib.pyplot as plt
import time, os, shutil
import dv_processing as dv
from PIL import Image
import numpy as np
import helper_funcs as hf

# Data Preprocessing 

1. Create batches from the recordings. Each batch can be split into `batch_time`.

2. Random sample some `N` batches from each subject & letter. Retrieve 5%? of each batch?

3. Split into train and test datasets (80 - 20 split)

In [86]:
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k',
           'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 
           'v', 'w', 'x', 'y']


In [13]:
# MODE = "train"
MODE = "test"
for l in letters:
    df = pd.read_csv(f"../data/{MODE}/{l}.csv")
    df = df.reindex(columns=['x', 'y', 'timestamp', 'polarity'])
    starts = df[df['timestamp'] == 0].index

    DIR = f'../data/{MODE}/{l}'
    if not os.path.isdir(DIR):
        os.makedirs(DIR)

    for i, t in enumerate(starts):
        sample = df.iloc[t:starts[i+1]] if i+1 < len(starts) else df.iloc[t:]
        SAVE = f"{DIR}/test_{l}_{i:04}.bin"

        # Throw out samples with < 1000 events 
        if len(sample) < 1000:
            continue

        data = [tuple(row) for row in sample.to_numpy()]
        data = np.array(data, dtype=[('x', '<i8'), ('y', '<i8'), ('t', '<i8'), ('p', '<i8')])

        with open(SAVE, 'wb') as f:
            np.save(f, data)


In [87]:
DIR = f"/Users/ria/Documents/GitHub/COSC525_ASL_SNN/data/train"

for l in letters:

    D = f"{DIR}/{l}"

    files = [f for f in sorted(os.listdir(D))]

    # Shuffle indices - create train and test sets
    idx = np.arange(len(files))
    np.random.shuffle(idx)
    s = int(len(files) * 0.7)
    train = idx[:s]
    test = idx[s:]

    train = [f for i, f in enumerate(files) if i in train]
    test = [f for i, f in enumerate(files) if i in test]

    for i, f in enumerate(sorted(os.listdir(D))):

        if f in test:
            print(f)

    print('----------\n')

.DS_Store
a_0011.bin
a_0019.bin
a_0024.bin
a_0025.bin
a_0026.bin
a_0029.bin
----------

b_0002.bin
b_0004.bin
b_0005.bin
b_0006.bin
b_0017.bin
b_0021.bin
b_0027.bin
----------

.DS_Store
c_0007.bin
c_0012.bin
c_0019.bin
c_0020.bin
c_0021.bin
c_0027.bin
----------

d_0002.bin
d_0010.bin
d_0011.bin
d_0014.bin
d_0016.bin
d_0024.bin
d_0027.bin
----------

e_0013.bin
e_0016.bin
e_0017.bin
e_0020.bin
e_0023.bin
e_0024.bin
e_0029.bin
----------

f_0000.bin
f_0001.bin
f_0005.bin
f_0013.bin
f_0016.bin
f_0019.bin
f_0029.bin
----------

.DS_Store
g_0004.bin
g_0005.bin
g_0010.bin
g_0012.bin
g_0017.bin
----------

.DS_Store
h_0003.bin
h_0008.bin
h_0015.bin
h_0019.bin
h_0020.bin
h_0025.bin
----------

i_0006.bin
i_0007.bin
i_0008.bin
i_0011.bin
i_0018.bin
i_0019.bin
i_0028.bin
----------

k_0001.bin
k_0010.bin
k_0021.bin
k_0023.bin
k_0024.bin
k_0028.bin
k_0029.bin
----------

.DS_Store
l_0000.bin
l_0007.bin
l_0016.bin
l_0020.bin
l_0026.bin
l_0027.bin
----------

.DS_Store
m_0001.bin
m_0005.bin
m_001

In [103]:
df = pd.DataFrame(columns=['file', 'label'])

labs = 0
prev = []

for f in sorted(os.listdir("../data/train")):

    label = f.split('_')[0]
    
    if label not in prev:
        prev.append(label)
        labs += 1

    path = f"./data/train/{f}"

    data = pd.DataFrame({'file': path, 'label': [labs-1]})
    df = pd.concat([df, data])

df.to_csv("../data/train/train_data.csv", index=False)

In [99]:
df

,file,label
0,./data/train/a_0000.bin,0
0,./data/train/a_0001.bin,0
0,./data/train/a_0003.bin,0
0,./data/train/a_0004.bin,0
0,./data/train/a_0007.bin,0
...,...,...
0,./data/train/y_0017.bin,23
0,./data/train/y_0019.bin,23
0,./data/train/y_0020.bin,23
0,./data/train/y_0022.bin,23


In [105]:
df = pd.DataFrame(columns=['file', 'label'])

labs = 0
prev = []

for f in sorted(os.listdir("../data/test")):

    label = f.split('_')[0]
    
    if label not in prev:
        prev.append(label)
        labs += 1

    path = f"./data/test/{f}"

    data = pd.DataFrame({'file': path, 'label': [labs-1]})
    df = pd.concat([df, data])

df.to_csv("../data/test/test_data.csv", index=False)

In [110]:
trainset = pd.read_csv("../data/train_data.csv")
testset = pd.read_csv("../data/test_data.csv")

print(f"Trainset: {len(trainset)} items")
print(f"Testset: {len(testset)} items")
print('-----')
print(f"Total data: {len(trainset) + len(testset)} items")

Trainset: 497 items
Testset: 214 items
-----
Total data: 711 items
